<a href="https://colab.research.google.com/github/devayache/AluraVoz/blob/main/AluraVoz.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Alura Voz - Taxa de Churn


![Alura Voz](https://raw.githubusercontent.com/devayache/AluraVoz/main/68747470733a2f2f692e696d6775722e636f6d2f6a6e376b6d386f2e706e67.png)

## O que é a Taxa de Churn?

Para todas as empresa o fundamental é aumentar o seu número de clientes, por isso dizem que a propaganda é a alma do negócio, porém muitas vezes a empresa não se atenta a manter os clientes que já tem. Evitar os cancelamentos é tão importante quanto capitar novos clientes. Sendo assim uma das métricas usadas para medir o nível de cancelamento de uma empresa é a taxa de Churn, essa taxa é medida sempre em um determinado período, sendo assim basta dividir a quantidade de clientes que cancelaram no período pela quantidade de clientes que havia iniciado neste período.

## $$ \text{Taxa de Churn} =  \frac{\text{Quantidade de cancelamento no Período}}{\text{Total de clientes no Período}}$$

Assim pode-se concluir que o aumento do faturamento está ligado diretamente a uma taxa baixa de Churn, uma taxa de Churn considerada boa deve ficar abaixo do 10%, claro que quanto menor melhor. 
Mas e como fazer esta taxa baixar ? Bem, para isso é preciso de um estudo mais detalhado dos clientes, observar como eles estão reagindo e tentar prever um perfil para os possíveis cancelamentos.
O banco da dados a seguir vem de uma API e tem seu formato Json, para isso então será utilizada algumas técnicas de organização dos dados para que se possa ter uma melhor ideia dos dados.


# Carregandos dados

In [1]:
# importação das bibliotecas
import pandas as pd
import numpy as np
import matplotlib as plt

In [2]:
# configurações inicias
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_colwidth', None)

In [3]:
clientes = pd.read_json('https://raw.githubusercontent.com/devayache/AluraVoz/main/Telco-Customer-Churn.json')
clientes.head()

,customerID,Churn,customer,phone,internet,account
0,0002-ORFBO,No,"{'gender': 'Female', 'SeniorCitizen': 0, 'Partner': 'Yes', 'Dependents': 'Yes', 'tenure': 9}","{'PhoneService': 'Yes', 'MultipleLines': 'No'}","{'InternetService': 'DSL', 'OnlineSecurity': 'No', 'OnlineBackup': 'Yes', 'DeviceProtection': 'No', 'TechSupport': 'Yes', 'StreamingTV': 'Yes', 'StreamingMovies': 'No'}","{'Contract': 'One year', 'PaperlessBilling': 'Yes', 'PaymentMethod': 'Mailed check', 'Charges': {'Monthly': 65.6, 'Total': '593.3'}}"
1,0003-MKNFE,No,"{'gender': 'Male', 'SeniorCitizen': 0, 'Partner': 'No', 'Dependents': 'No', 'tenure': 9}","{'PhoneService': 'Yes', 'MultipleLines': 'Yes'}","{'InternetService': 'DSL', 'OnlineSecurity': 'No', 'OnlineBackup': 'No', 'DeviceProtection': 'No', 'TechSupport': 'No', 'StreamingTV': 'No', 'StreamingMovies': 'Yes'}","{'Contract': 'Month-to-month', 'PaperlessBilling': 'No', 'PaymentMethod': 'Mailed check', 'Charges': {'Monthly': 59.9, 'Total': '542.4'}}"
2,0004-TLHLJ,Yes,"{'gender': 'Male', 'SeniorCitizen': 0, 'Partner': 'No', 'Dependents': 'No', 'tenure': 4}","{'PhoneService': 'Yes', 'MultipleLines': 'No'}","{'InternetService': 'Fiber optic', 'OnlineSecurity': 'No', 'OnlineBackup': 'No', 'DeviceProtection': 'Yes', 'TechSupport': 'No', 'StreamingTV': 'No', 'StreamingMovies': 'No'}","{'Contract': 'Month-to-month', 'PaperlessBilling': 'Yes', 'PaymentMethod': 'Electronic check', 'Charges': {'Monthly': 73.9, 'Total': '280.85'}}"
3,0011-IGKFF,Yes,"{'gender': 'Male', 'SeniorCitizen': 1, 'Partner': 'Yes', 'Dependents': 'No', 'tenure': 13}","{'PhoneService': 'Yes', 'MultipleLines': 'No'}","{'InternetService': 'Fiber optic', 'OnlineSecurity': 'No', 'OnlineBackup': 'Yes', 'DeviceProtection': 'Yes', 'TechSupport': 'No', 'StreamingTV': 'Yes', 'StreamingMovies': 'Yes'}","{'Contract': 'Month-to-month', 'PaperlessBilling': 'Yes', 'PaymentMethod': 'Electronic check', 'Charges': {'Monthly': 98.0, 'Total': '1237.85'}}"
4,0013-EXCHZ,Yes,"{'gender': 'Female', 'SeniorCitizen': 1, 'Partner': 'Yes', 'Dependents': 'No', 'tenure': 3}","{'PhoneService': 'Yes', 'MultipleLines': 'No'}","{'InternetService': 'Fiber optic', 'OnlineSecurity': 'No', 'OnlineBackup': 'No', 'DeviceProtection': 'No', 'TechSupport': 'Yes', 'StreamingTV': 'Yes', 'StreamingMovies': 'No'}","{'Contract': 'Month-to-month', 'PaperlessBilling': 'Yes', 'PaymentMethod': 'Mailed check', 'Charges': {'Monthly': 83.9, 'Total': '267.4'}}"


Como dito anteriormente os dados estão em um formato Json o que necessita de uma atenção especial para o tratamento dos mesmo, tendo em vista que o DataFrame inicial é possível observar que as colunas trazem mais informações, ou seja, trazem outras variáveis a serem estudas, para isso então é preciso “explodir” cada uma das colunas para se ter uma melhor visão dos dados.

In [4]:
# Normalizando as colunas
clientes_informacoes = pd.json_normalize(clientes['customer'])
clientes_telefone = pd.json_normalize(clientes['phone'])
clientes_internet = pd.json_normalize(clientes['internet'])
clientes_conta = pd.json_normalize(clientes['account'])

In [5]:
# dropando as colunas 
clientes.drop(columns=['customer'],inplace=True)
clientes.drop(columns=['phone'],inplace=True)
clientes.drop(columns=['internet'],inplace=True)
clientes.drop(columns=['account'],inplace=True)

In [6]:
print(f"Total de clientes fornecidos na tabela: {clientes.shape[0]}")

Total de clientes fornecidos na tabela: 7267


# Analisado as colunas separadamente

Aqui as novas colunas geradas serão analisadas separadamente, começando pela colunas gerada pela antiga coluna `customer`

## Variaveis da coluna `customer`

In [36]:
clientes_informacoes.head()

,gender,SeniorCitizen,Partner,Dependents,tenure
0,Female,0,Yes,Yes,9
1,Male,0,No,No,9
2,Male,0,No,No,4
3,Male,1,Yes,No,13
4,Female,1,Yes,No,3


### Dicionário da coluna `customer`.
Seque o dicionário da coluna `customer`. Para um processo de tradução ela será chamada de `informações`, pois é justamente isso que traz dos usuários.

\
`gender`: gênero (masculino e feminino)

`SeniorCitizen`: informação sobre um cliente ter ou não idade igual ou maior que 65 anos
`Partner`: se o cliente possui ou não um parceiro ou parceira

`Dependents`: se o cliente possui ou não dependentes

`tenure`: meses de contrato do cliente


In [8]:
clientes_informacoes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7267 entries, 0 to 7266
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   gender         7267 non-null   object
 1   SeniorCitizen  7267 non-null   int64 
 2   Partner        7267 non-null   object
 3   Dependents     7267 non-null   object
 4   tenure         7267 non-null   int64 
dtypes: int64(2), object(3)
memory usage: 284.0+ KB


### Tipos de variáveis

* Variáveis categóricas:
  * `gender`
  * `SeniorCitizen`
  * `Partner`
  * `Dependents`
* Variáveis quantitativa:
  * `ternure`


\
Observação: Apesar de a variável `SeniorCitizen` aparecer como inteira nas informações ela simplesmente informa se o clientes possui mais de 65 anos ou não, ou seja, dando uma característica ao cliente e por isso ela será tratada como uma variável categórica.



### Verificando as informações em cada coluna

In [9]:
clientes_informacoes['gender'].unique()

array(['Female', 'Male'], dtype=object)

In [10]:
clientes_informacoes['SeniorCitizen'].unique()

array([0, 1])

In [11]:
clientes_informacoes['Partner'].unique()

array(['Yes', 'No'], dtype=object)

In [12]:
clientes_informacoes['Dependents'].unique()

array(['Yes', 'No'], dtype=object)

In [13]:
clientes_informacoes['tenure'].unique()

array([ 9,  4, 13,  3, 71, 63,  7, 65, 54, 72,  5, 56, 34,  1, 45, 50, 23,
       55, 26, 69, 11, 37, 49, 66, 67, 20, 43, 59, 12, 27,  2, 25, 29, 14,
       35, 64, 39, 40,  6, 30, 70, 57, 58, 16, 32, 33, 10, 21, 61, 15, 44,
       22, 24, 19, 47, 62, 46, 52,  8, 60, 48, 28, 41, 53, 68, 51, 31, 36,
       17, 18, 38, 42,  0])

Ao observar os valores de cada uma das colunas que trazem as informações dos clientes verifica-se que, as variáveis estão de acordo com que se é esperado, porém é preciso fazer uma ressalva para o valor `0`, que se encontra na variável `tenure`, pois ele pode se tratar de uma inconsistência ou somente de novos clientes com menos de um mês de contrato vigente, contudo esses dados serão mais facilmente analisados quando as tabelas forem juntas pois assim se poderá ter uma noção com os valores totais gastos por esses clientes.


## Variaveis da coluna `Phone`

In [37]:
clientes_telefone.head()

,PhoneService,MultipleLines
0,Yes,No
1,Yes,Yes
2,Yes,No
3,Yes,No
4,Yes,No


### Dicionário da coluna `phone	`.

Seque o dicionário da coluna `phone`. Para um processo de tradução ela será chamada de `telefone`, pois é justamente disso que se trat.

\
`PhoneService`: assinatura de serviço telefônico

`MultipleLines`: assinatura de mais de uma linha de telefone


In [15]:
clientes_telefone.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7267 entries, 0 to 7266
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   PhoneService   7267 non-null   object
 1   MultipleLines  7267 non-null   object
dtypes: object(2)
memory usage: 113.7+ KB


### Tipos de variáveis

* Variáveis categóricas:
  * `PhoneService`
  * `MultpleLines`

Aqui ambas as variáveis são categóricas

### Verificando as informações em cada coluna

In [16]:
clientes_telefone['PhoneService'].unique()

array(['Yes', 'No'], dtype=object)

In [17]:
clientes_telefone['MultipleLines'].unique()

array(['No', 'Yes', 'No phone service'], dtype=object)

Aqui é preciso fazer uma rápida verificação para ver se não a inconsistências, pois na variável `MultiplesLines` uma das opções é `No phone service` e ela não pode aparecer quando o coluna `PhoneService` for igual a `Yes`.

In [34]:
clientes_telefone[clientes_telefone['MultipleLines'] == 'No phone service'].value_counts()

PhoneService  MultipleLines   
No            No phone service    707
dtype: int64

É possível verificar que 707 clientes não possuem serviço de internet e todos eles têm a opção `No` marcada na coluna `PhoneService`, sendo assim averiguado que não existe esse inconsistência

## Variaveis da coluna `internet`

In [38]:
clientes_internet.head()

,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies
0,DSL,No,Yes,No,Yes,Yes,No
1,DSL,No,No,No,No,No,Yes
2,Fiber optic,No,No,Yes,No,No,No
3,Fiber optic,No,Yes,Yes,No,Yes,Yes
4,Fiber optic,No,No,No,Yes,Yes,No


### Dicionário da coluna `internet`.

Seque o dicionário da coluna `internet`.

\
`InternetService`: assinatura de um provedor internet

`OnlineSecurity`: assinatura adicional de segurança online

`OnlineBackup`: assinatura adicional de backup online

`DeviceProtection`: assinatura adicional de proteção no dispositivo

`TechSupport`: assinatura adicional de suporte técnico, menos tempo de espera

`StreamingTV`: assinatura de TV a cabo

`StreamingMovies`: assinatura de streaming de filmes

In [39]:
clientes_internet.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7267 entries, 0 to 7266
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   InternetService   7267 non-null   object
 1   OnlineSecurity    7267 non-null   object
 2   OnlineBackup      7267 non-null   object
 3   DeviceProtection  7267 non-null   object
 4   TechSupport       7267 non-null   object
 5   StreamingTV       7267 non-null   object
 6   StreamingMovies   7267 non-null   object
dtypes: object(7)
memory usage: 397.5+ KB


### Tipos de variáveis

* Variáveis categóricas:
  * `InternetService`
  * `OnlineSecurity`
  * `OnlineBackup`
  * `DeviceProtection`
  * `TechSupport`
  * `StreamingTV`
  * `StreamingMovies`

Todas as variáveis são categóricas


### Verificando as informações em cada coluna

In [53]:
for i in clientes_internet.columns:
  print(f'{i} ==> {clientes_internet[i].unique()}\n')

InternetService ==> ['DSL' 'Fiber optic' 'No']

OnlineSecurity ==> ['No' 'Yes' 'No internet service']

OnlineBackup ==> ['Yes' 'No' 'No internet service']

DeviceProtection ==> ['No' 'Yes' 'No internet service']

TechSupport ==> ['Yes' 'No' 'No internet service']

StreamingTV ==> ['Yes' 'No' 'No internet service']

StreamingMovies ==> ['No' 'Yes' 'No internet service']



Análoga ao que foi feito no serviço de telefone é preciso verificar se as opções `No internet sevice` aparecem quando o na coluna `InternetSevice` aparece como `Yes`, pois se isso acontecer será considerado uma inconsistência

In [127]:
for i in clientes_internet.columns[1:]:
  selecao = (clientes_internet['InternetService'] == 'No') & (clientes_internet[i] == 'No internet service')
  print(f'{i} ==> {clientes[selecao].shape[0]}')

OnlineSecurity ==> 1581
OnlineBackup ==> 1581
DeviceProtection ==> 1581
TechSupport ==> 1581
StreamingTV ==> 1581
StreamingMovies ==> 1581


# Apoio

### Tradução dos nomes das colunas:

* `gender` = `gênero`
* `SeniorCitizen` = `idoso` 
* `Partner` = `cônjuge`
* `Dependents` = `dependentes`
* `tenure` =  `permanência`

In [16]:
# dicionário com as traduções
translate = {
    'Female': 'Mulher', 
    'Male': 'Homem',
    'Yes': 'Sim', 
    'No': 'Nao', 
    'No phone service' : 'Sem Serviço de telefone', 
    'No internet service' : 'Sem Serviço de internet',
    'One year': 'Um ano', 
    'Month-to-month': 'Mensal', 
    'Two year' : 'Dois anos',
    'Mailed check': 'Cheque por Correio', 
    'Electronic check': 'Cheque eletronico', 
    'Credit card (automatic)': 'Cartao de credito',
    'Bank transfer (automatic)': 'Transferencia bancaria'
}


In [ ]:
#aproveitando para traduzir o nome das colunas 
clientes_informacoes.rename(columns = {
    'gender': 'gênero', 
    'SeniorCitizen': 'idoso', 
    'Partner': 'cônjuge',
    'Dependents': 'dependentes',
    'tenure': 'permanência'
    }, inplace=True)
clientes_informacoes.columns.name = 'Id'
clientes_informacoes

In [ ]:
clientes_informacoes.replace(to_replace=translate, inplace=True)
clientes_informacoes

In [ ]:
taxa_churn =  (clientes[clientes['Churn'] =="Yes"].shape[0]/clientes.shape[0]*100)
print(f"A taxa de Churn na primeira análise: {taxa_churn:.2f}%")

In [ ]:
clientes[(clientes['Churn']=="Yes") & (clientes['SeniorCitizen']==1)].shape[0]
# df[(df.sex=='Male') & (df.smoker=='Yes')]

In [ ]:
# juntando as colunas
clientes = pd.concat([clientes, clientes_informacoes, clientes_telefone, clientes_internet, clientes_conta], axis=1)
clientes.head()